In [ ]:
import numpy as np
import pandas as pd

def ew_cov_corr_normalized(df: pd.DataFrame, lam: float = 0.97, window: int | None = None):
    X = df.to_numpy(copy=False)
    if window is not None:
        X = X[-window:, :]
    n, k = X.shape

    # normalized weights: newest obs has exponent 0
    exponents = np.arange(n-1, -1, -1, dtype=float)
    w_raw = (1.0 - lam) * (lam ** exponents)
    w = w_raw / w_raw.sum()
    w = w.reshape(-1, 1)

    # mean over the same window (can switch to simple mean if needed)
    mu = (w.T @ X) / w.sum()
    # mu = X.mean(axis=0, keepdims=True)   # simple mean alternative

    XC = X - mu
    cov = (XC.T * w.ravel()) @ XC

    std = np.sqrt(np.diag(cov))
    denom = np.outer(std, std)
    with np.errstate(invalid="ignore", divide="ignore"):
        corr = cov / denom
    corr[np.isnan(corr)] = 0.0

    cov_df = pd.DataFrame(cov, index=df.columns, columns=df.columns)
    corr_df = pd.DataFrame(corr, index=df.columns, columns=df.columns)
    return cov_df, corr_df

In [2]:
def ew_covariance_21(df: pd.DataFrame, lam: float = 0.97, window: int | None = None) -> pd.DataFrame:
    """2.1: EW Covariance with lambda=lam (uses your normalized version)."""
    cov, _ = ew_cov_corr_normalized(df, lam=lam, window=window)
    return cov

def ew_correlation_22(df: pd.DataFrame, lam: float = 0.94, window: int | None = None) -> pd.DataFrame:
    """2.2: EW Correlation with lambda=lam (uses your normalized version)."""
    _, corr = ew_cov_corr_normalized(df, lam=lam, window=window)
    return corr

def covariance_23(df: pd.DataFrame,
                  lam_var: float = 0.97,
                  lam_corr: float = 0.94,
                  window: int | None = None) -> pd.DataFrame:
    """
    2.3: Covariance with EW Variance (lam_var) and EW Correlation (lam_corr):
         Σ = D * R * D, where D = diag(EW std with lam_var), R = EW corr with lam_corr.
    """
    # EW variance (from EW covariance with lam_var)
    cov_var = ew_covariance_21(df, lam=lam_var, window=window)
    std = np.sqrt(np.diag(cov_var))
    D = np.diag(std)

    # EW correlation with lam_corr
    R = ew_correlation_22(df, lam=lam_corr, window=window).values

    Sigma = D @ R @ D
    return pd.DataFrame(Sigma, index=df.columns, columns=df.columns)

In [3]:
df = pd.read_csv("../testfiles/data/test2.csv")   

In [4]:
## Test2.1     
cov_21  = ew_covariance_21(df, lam=0.97)
print("EW Covariance_lambda=0.97:")
print(cov_21)

EW Covariance_lambda=0.97:
          x1        x2        x3        x4        x5
x1  0.848630  0.120043  0.179149  0.083942  0.055432
x2  0.120043  1.079589  0.024682  0.115124 -0.429611
x3  0.179149  0.024682  0.736457  0.133765  0.069033
x4  0.083942  0.115124  0.133765  0.867932  0.112788
x5  0.055432 -0.429611  0.069033  0.112788  1.137927


In [5]:
## Test2.2    
corr_22 = ew_correlation_22(df, lam=0.94)
print("EW Correlation_lambda=0.94:")
print(corr_22)

EW Correlation_lambda=0.94:
          x1        x2        x3        x4        x5
x1  1.000000  0.084787  0.190100  0.129045  0.070706
x2  0.084787  1.000000 -0.077432  0.202228 -0.442546
x3  0.190100 -0.077432  1.000000  0.198267  0.102093
x4  0.129045  0.202228  0.198267  1.000000  0.119541
x5  0.070706 -0.442546  0.102093  0.119541  1.000000


In [6]:
## Test2.3
cov_23  = covariance_23(df, lam_var=0.97, lam_corr=0.94)
print("Covariance with EW Variance (l=0.97), EW Correlation (l=0.94)")
print(cov_23)

Covariance with EW Variance (l=0.97), EW Correlation (l=0.94)
          x1        x2        x3        x4        x5
x1  0.848630  0.081156  0.150285  0.110750  0.069482
x2  0.081156  1.079589 -0.069043  0.195755 -0.490507
x3  0.150285 -0.069043  0.736457  0.158514  0.093460
x4  0.110750  0.195755  0.158514  0.867932  0.118800
x5  0.069482 -0.490507  0.093460  0.118800  1.137927
